<a href="https://colab.research.google.com/github/mehdiabbasidev/darsman-machine-learning/blob/main/InitialOperation_FeatureSelection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Dataset download link:
https://drive.google.com/file/d/1wMqhiBq_GGF_5nYKbbMC2njSSjcLU8YD/view?usp=sharing

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/datasets/FeatureSelectionDatasets/feature_selection_dataset1.csv')
data.head()

In [ ]:
data.shape

In [ ]:
data['feature_16'].value_counts()
data['feature_17'].value_counts()

In [ ]:
features_to_encode = ['feature_16', 'feature_17']
le = LabelEncoder()
for feature in features_to_encode:
    data[feature] = le.fit_transform(data[feature])
data.head()

### Removing constant features

In [ ]:
data.shape

In [ ]:
data.nunique()

In [ ]:
X=data.drop(labels=['target'], axis=1)
X.shape

In [ ]:
constant_features = [col for col in X.columns if X[col].nunique() <= 1]
constant_features

In [ ]:
data=data.drop(labels=constant_features, axis=1)
data.head()

### Remove quasi-constant features


In [ ]:
data.shape

In [ ]:
data.dtypes

In [ ]:
X=data.drop(labels=['target'], axis=1)
X.shape

In [ ]:
quasi_constants = []
for col in X.columns:
    feature_freq= X[col].value_counts(normalize=True).sort_values(ascending=False).values[0]
    # print(feature_freq)
    if feature_freq > 0.990:
        quasi_constants.append(col)
quasi_constants

In [ ]:
data=data.drop(labels=quasi_constants, axis=1)
data.head()

### Remove duplicated features

In [ ]:
data.shape

In [ ]:
X=data.drop(['target'],axis=1)
X.shape

In [ ]:
duplicate_columns = []
for i in range(len(X.columns)):
    col1 = X.columns[i]
    for j in range(i + 1, len(X.columns)):
        col2 = X.columns[j]
        if X[col1].equals(X[col2]):
            duplicate_columns.append(col2)
duplicate_columns

In [ ]:
data=data.drop(labels=duplicate_columns, axis=1)
data.head()

In [ ]:
data.to_csv('out.csv', index=False)

### Remove features with high correlation

In [ ]:
data.shape

In [ ]:
X=data.drop(['target'],axis=1)
X.shape

In [ ]:
corrmat = X.corr(method='pearson')
cmap = sns.diverging_palette(220, 20, as_cmap=True)
fig, ax = plt.subplots()
fig.set_size_inches(8,8)
sns.heatmap(corrmat, cmap=cmap)

In [ ]:
corr_matrix = X.corr().abs()
print(corr_matrix)

In [ ]:
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
corr_columns = [column for column in upper.columns if any(upper[column] > 0.9)]
corr_columns

In [ ]:
data=data.drop(labels=corr_columns, axis=1)
data.head()

### Select features with high Mutual Informaion

In [ ]:
data.shape

In [ ]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif

X = data.drop('target', axis=1)
y = data['target']

if (X < 0).any().any():
    X = X + abs(X.min().min())

selector = SelectKBest(score_func=mutual_info_classif, k=5)
selector.fit(X, y)
selected_features = selector.get_support(indices=True)
selected_feature_names = X.columns[selected_features]

print("Selected features:", selected_feature_names)

X_selected = X[selected_feature_names]
X_selected